## Equilibrium and stability basic analysis
### (Canard sizing)

Nemo human powered hydrofoil

June 2016 - Gustavo Violato, Diego Montero, Fernando Valentini

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
%matplotlib notebook

### Scilab prelim_perf.sce results

#### 2.00m wing

>Results:
>
>Chosen wind span[m]/aspect ratio[-]: 2.00, 18.0
>
>-Rider----------<br>
Name     : Gustavo<br>
Mass [kg]: 87.0<br>
Power [W]: 250.0<br>
>
>-Wing-----------<br>
Wing area [m^2]: 0.222<br>
Root Chord  [m]: 0.153<br>
Tip Chord   [m]: 0.069<br>
>
>-Performance----<br>
Take-off speed [m/s]: 3.0<br>
Take-off power   [W]: 150.0<br>
Cruise Speed   [m/s]: 4.6<br>
Range           [km]: 16.62<br>
Speed@750W     [m/s]: 7.2<br>
Min power        [W]: 146.9<br>
Speed@MinPwr   [m/s]: 2.7<br>

<img src='perf_curves_S2_AR18.gif'>

#### 1.80m wing

>Results:
>
>Chosen wind span[m]/aspect ratio[-]: 1.80, 17.0
>
>-Rider----------<br>
Name     : Gustavo<br>
Mass [kg]: 87.0<br>
Power [W]: 250.0<br>
>
>-Wing-----------<br>
Wing area [m^2]: 0.191<br>
Root Chord  [m]: 0.146<br>
Tip Chord   [m]: 0.066<br>
>
>-Performance----<br>
Take-off speed [m/s]: 3.3<br>
Take-off power   [W]: 172.7<br>
Cruise Speed   [m/s]: 4.6<br>
Range           [km]: 16.56<br>
Speed@750W     [m/s]: 7.4<br>
Min power        [W]: 168.8<br>
Speed@MinPwr   [m/s]: 2.9<br>

<img src='perf_curves_b1p8_AR17.gif'>

In [150]:
# Constants definitions
grav = 9.80665 # gravity [m/s^2]
rho_w = 1020 # (sea) water density [kg/m^3]

V_cruise = 4.5 # cruise speed [m/s]
Q_cruise = 0.5*rho_w*V_cruise**2

mass     = 101  # total vehicle mass [kg]
delta_cg = 0.0  # forward distance of actual CG pos to nominal CG pos

# Wing geometry and properties
wing_pos = 0.11    # horizontal distance from nominal CG pos to wing quarter chord
xw0      = wing_pos + delta_cg
h_cg_w   = 0.985   # vertical distance from nominal CG pos to wing quarter chord
r_w      = np.sqrt(xw0**2 + h_cg_w**2)
delta_w  = np.arctan(xw0/h_cg_w)
Sref     = 0.191 # (wing) reference area [m^2]
cr       = 0.146
ctip     = 0.066
cma      = 2./3*(cr+ctip-cr*ctip/(cr+ctip))
Cla_w    = 6.277/(1+(6.277/(np.pi*17))*(1+0.03))
Cl0_w    = 0.4/6.227*Cla_w
Cm_w     = -0.075
i_w      = 2.5*np.pi/180.

print "Wing Props:"
print "-----------"
print "S_w (Sref): {:.3f} m^2".format(Sref)
print "cma_w (cma): {:.3f} m".format(cma)
print "r_w: {:.3f} m".format(r_w)
print "delta_w: {:.2f} deg".format(delta_w*180/np.pi)
print "Cla_w: {:.2f}".format(Cla_w)
print "Cl0_w: {:.2f}".format(Cl0_w)

# Canard geometry and properties
C_factor = 0.08
AR_c = 7
S_c  = Sref*C_factor
b_c = np.sqrt(AR_c*S_c)
c_c = S_c/b_c
x_cw_qc = 1.515 + 0.25*(cr-c_c) # distance between canard and wing quarter chord
xc0 = x_cw_qc - xw0
Cla_c    = 5.988/(1+(5.988/(np.pi*AR_c))*(1+0.17))
Cl0_c    = 0.4/5.988*Cla_c
Cm_c     = -0.08

print "\nCanard Props:"
print "-------------"
print "S_c: {:.3f} m^2".format(S_c)
print "b_c: {:.3f} m".format(b_c)
print "cma_c: {:.3f} m".format(c_c)
print "xc0: {:.3f} m".format(xc0)
print "Cla_c: {:.2f}".format(Cla_c)
print "Cl0_c: {:.2f}".format(Cl0_c)

# Keel geometry and properties
d_k = 0.030 # Keel tube diameter
t_k = 0.15  # Keel fairing profile relative thickness
c_k = d_k/t_k # Keel chord
S_k = c_k*0.5 # Keel submerged area
h_cp_k = 0.5  # Keel drag arm relative to nominal cg pos
Cd_k = 0.007  # Keel profile drag Coef

print "\nKeel Props:"
print "-----------"
print "c_k: {:.3f} m".format(c_k)
print "S_k: {:.3f} m^2".format(S_k)
print "h_cp_k: {:.3f} m".format(h_cp_k)
print "Cd_k: {:.3f}".format(Cd_k)

Wing Props:
-----------
S_w (Sref): 0.191 m^2
cma_w (cma): 0.111 m
r_w: 0.991 m
delta_w: 6.37 deg
Cla_w: 5.60
Cl0_w: 0.36

Canard Props:
-------------
S_c: 0.015 m^2
b_c: 0.327 m
cma_c: 0.047 m
xc0: 1.430 m
Cla_c: 4.54
Cl0_c: 0.30

Keel Props:
-----------
c_k: 0.200 m
S_k: 0.100 m^2
h_cp_k: 0.500 m
Cd_k: 0.007


In [151]:
CLeq = mass*grav/(Q_cruise*Sref)

CF_a  = (C_factor*Cla_c + Cla_w)
CF_dp = (C_factor*Cla_c)
CF_0  = CLeq - C_factor*Cl0_c-Cl0_w-Cla_w*i_w

CM_a2 = r_w/cma*Cla_w
CM_a  = C_factor*xc0/cma*Cla_c + r_w/cma*(Cl0_w+Cla_w*(i_w-delta_w))
CM_dp = C_factor*xc0/cma*Cla_c
CM_0  = S_k/Sref*h_cp_k/cma*Cd_k-C_factor*c_c/cma*Cm_c-Cm_w-C_factor*xc0/cma*Cl0_c+r_w/cma*delta_w*(Cl0_w+Cla_w*i_w)

def equilibrium(p):
    alpha, dp = p
    eq1 = CF_a*alpha + CF_dp*dp - CF_0
    eq2 = CM_a2*alpha**2 + CM_a*alpha + CM_dp*dp - CM_0
    return (eq1,eq2)
    
a, dp = fsolve(equilibrium,(4*np.pi/180.,5*np.pi/180.))

Cl_w = Cl0_w + (a+i_w)*Cla_w
L_w = Q_cruise*Sref*Cl_w/grav
Cl_c = Cl0_c + (a+dp)*Cla_c
L_c = Q_cruise*S_c*Cl_c/grav

Xn = x_cw_qc/(1+(Cla_w/(C_factor*Cla_c)))

print "Equilibrium conditions for\nMass {:.1f}kg, V {:.1f} m/s, Cg {:.3f} m ahead of wing-1/4c".format(mass,V_cruise,xw0)
print "---------------------------"
print "alpha: {:.3f}deg; dp:{:.3f}deg".format(a*180/np.pi, dp*180/np.pi)
print "Wing Cl: {:.2f} (Wing Lift: {:.1f}kgf)".format(Cl_w,L_w)
print "Canard Cl: {:.2f} (Canard Lift: {:.1f}kgf)".format(Cl_c,L_c)
print "Neutral Point: {:.3f}m ahead of wing-1/4c".format(Xn)
print "CG Pos: {:.3f}m ahead of wing-1/4c".format(r_w*np.sin(delta_w-a))

Equilibrium conditions for
Mass 101.0kg, V 4.5 m/s, Cg 0.110 m ahead of wing-1/4c
---------------------------
alpha: -1.561deg; dp:5.720deg
Wing Cl: 0.45 (Wing Lift: 90.8kgf)
Canard Cl: 0.63 (Canard Lift: 10.2kgf)
Neutral Point: 0.094m ahead of wing-1/4c
CG Pos: 0.137m ahead of wing-1/4c


In [111]:
print L_c*xc0*grav, L_w*r_w*(delta_w-a)*grav
print Q_cruise*Cm_w*Sref*cma, Q_cruise*Cm_c*S_c*c_c
print Q_cruise*Cd_k*S_k*h_cp_k
L_c*xc0*grav-L_w*r_w*(delta_w-a)*grav+Q_cruise*Cm_w*Sref*cma+Q_cruise*Cm_c*S_c*c_c-Q_cruise*Cd_k*S_k*h_cp_k

60.0262945061 48.9316167531
-8.83361941726 -0.317193335756
1.943865


-2.4424906541753444e-15

In [141]:
print 0.178-0.25*cr
print h_cg_w
print CLeq

0.1415
0.985
0.502126888891


In [111]:
Xc_avl = Xt-0.25*(cr-c_c)
Xl_avl = Xc_avl - c_c
print "Xc_avl: {}".format(Xc_avl)
print "Xl_avl: {}".format(Xl_avl)

Xc_avl: 1.56507575758
Xl_avl: 1.51877272727


In [123]:
b_l = 0.30
c_l = 0.06
A_l = c_l*b_l
AR_l = b_l**2/(A_l)
print b_l, c_l, A_l, AR_l

0.3 0.06 0.018 5.0


In [1]:
FACTOR = 1.1

rho_w = 1020
mu_w = 1e-3
Vel = 4.5

diam_q = 28

t_q = [0.12, 0.15, 0.18]
Cd0 = [0.0054, 0.0068, 0.0084]

for i,t in enumerate(t_q):
    cq = diam_q/t
    Re = rho_w*Vel*cq/mu_w/1000  
    Dq = 0.5*1020*4**2*Cd0[i]*cq*0.5/1000*FACTOR
    print "Espessura relativa: {:.2f}".format(t)
    print "Corda [mm]: {:.1f}".format(cq)
    print "Reynolds: {:.0f}".format(Re)
    print "Arrasto [kg]: {:.3f}".format(Dq/9.80665)
    print "\n"

Espessura relativa: 0.12
Corda [mm]: 233.3
Reynolds: 1071000
Arrasto [kg]: 0.577


Espessura relativa: 0.15
Corda [mm]: 186.7
Reynolds: 856800
Arrasto [kg]: 0.581


Espessura relativa: 0.18
Corda [mm]: 155.6
Reynolds: 714000
Arrasto [kg]: 0.598




In [20]:
from scipy.integrate import simps